<a href="https://colab.research.google.com/github/simonacarusooo/Algorithm-for-Massive-Datasets/blob/main/AMD_PROJECT_LINK_ANALYSIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**DOWNLOADING AND IMPORTING THE DATASET**

In [ ]:
import csv
import os
import pandas as pd   

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "simonacaruso99"
os.environ['KAGGLE_KEY'] = "d2755b2b7f8e067ba39d3b46b0cafddd"
!kaggle datasets download -d cynthiarempel/amazon-us-customer-reviews-dataset

amazon-us-customer-reviews-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import zipfile
import pandas as pd

#Reading the content of the zip file
zip_file = zipfile.ZipFile('/content/amazon-us-customer-reviews-dataset.zip')

#Dataframe structure
df = pd.DataFrame({'marketplace': pd.Series([], dtype='str'),
              'customer_id': pd.Series([], dtype='str'),
              'review_id': pd.Series([], dtype='str'),
              'product_id': pd.Series([], dtype='str'),
              'product_parent': pd.Series([], dtype='str'),
              'product_title': pd.Series([], dtype='str'),
              'product_category': pd.Series([], dtype='str'),
              'star_rating': pd.Series([], dtype='int'),
              'helpful_votes': pd.Series([], dtype='int'),
              'total_votes': pd.Series([], dtype='int'),
              'vine': pd.Series([], dtype='str'),
              'verified_purchase': pd.Series([], dtype='str'),
              'review_headline': pd.Series([], dtype='str'),
              'review_body': pd.Series([], dtype='str'),
              'review': pd.Series([], dtype='str'),
              'review_date': pd.Series([], dtype='float')})

#Selecting products categories tsv files
filenames = ['amazon_reviews_multilingual_US_v1_00.tsv','amazon_reviews_us_Apparel_v1_00.tsv','amazon_reviews_us_Automotive_v1_00.tsv', 'amazon_reviews_us_Baby_v1_00.tsv','amazon_reviews_us_Beauty_v1_00.tsv','amazon_reviews_us_Books_v1_02.tsv',
             'amazon_reviews_us_Camera_v1_00.tsv','amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tsv','amazon_reviews_us_Digital_Music_Purchase_v1_00.tsv','amazon_reviews_us_Digital_Software_v1_00.tsv','amazon_reviews_us_Digital_Video_Download_v1_00.tsv',
             'amazon_reviews_us_Digital_Video_Games_v1_00.tsv','amazon_reviews_us_Electronics_v1_00.tsv','amazon_reviews_us_Furniture_v1_00.tsv','amazon_reviews_us_Gift_Card_v1_00.tsv','amazon_reviews_us_Grocery_v1_00.tsv','amazon_reviews_us_Health_Personal_Care_v1_00.tsv',
             'amazon_reviews_us_Major_Appliances_v1_00.tsv','amazon_reviews_us_Mobile_Apps_v1_00.tsv','amazon_reviews_us_Mobile_Electronics_v1_00.tsv','amazon_reviews_us_Music_v1_00.tsv','amazon_reviews_us_Musical_Instruments_v1_00.tsv','amazon_reviews_us_Office_Products_v1_00.tsv',
             'amazon_reviews_us_Outdoors_v1_00.tsv','amazon_reviews_us_PC_v1_00.tsv','amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv','amazon_reviews_us_Pet_Products_v1_00.tsv','amazon_reviews_us_Shoes_v1_00.tsv','amazon_reviews_us_Software_v1_00.tsv','amazon_reviews_us_Sports_v1_00.tsv','amazon_reviews_us_Tools_v1_00.tsv',
             'amazon_reviews_us_Toys_v1_00.tsv','amazon_reviews_us_Video_DVD_v1_00.tsv','amazon_reviews_us_Video_Games_v1_00.tsv','amazon_reviews_us_Video_v1_00.tsv','amazon_reviews_us_Watches_v1_00.tsv','amazon_reviews_us_Wireless_v1_00.tsv']

#Selectin 1000 review for each category
for filename in filenames:
    with zip_file.open(filename) as f:
        size = 1000
        data = pd.read_csv(f, sep='\t', header=0, quoting=csv.QUOTE_NONE, nrows=size)
        data['review'] = data['review_headline'] + ' ' + data['review_body']
        df = df.append(data, ignore_index=True) 

print(df.shape)
df.head()


(37000, 16)


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review,review_date
0,US,53096384,R63J84G1LOX6R,1563890119,763187671,The Sandman Vol. 1: Preludes and Nocturnes,Books,4,0,1,N,N,ignore the review below,this is the first 8 issues of the series. it i...,ignore the review below this is the first 8 is...,1995-08-13
1,US,53096399,R1BALOA11Z06MT,1559947608,381720534,The 22 Immutable Laws of Marketing,Books,4,0,0,N,N,awesome,I've always been partial to immutable laws. Th...,awesome I've always been partial to immutable ...,1995-08-17
2,US,53096332,R1LLAY5W5PZUS4,0671701800,860650224,Contact,Books,5,1,2,N,N,Read the book. It's good.,This is a book about first contact with aliens...,Read the book. It's good. This is a book about...,1995-08-30
3,US,53096335,R3R9VTJ82FXECQ,0425132153,624269601,Good Omens,Books,5,0,0,N,N,Funniest book ever written about the Anti-Christ,This is quite possibly *the* funniest book I h...,Funniest book ever written about the Anti-Chri...,1995-09-11
4,US,51747709,R1P5J3FNBWTFXY,0517122707,161411385,A Confederacy of Dunces,Books,5,0,0,N,N,A winner that didn't last. Only a mothers love...,The story behind the book is almost better tha...,A winner that didn't last. Only a mothers love...,1995-10-17




**CREATION OF THE GRAPH**

In [ ]:
import networkx as nx
import numpy as np

In [ ]:
#Creation of an empty graph (no nodes and no edges)
G = nx.Graph()

#Taking the unique values of the column "product_id" and use them as nodes
for pr_id in df['product_id'].unique():
    G.add_node(pr_id)

for customer in df['customer_id'].unique():
    products = df[df['customer_id'] == customer]['product_id'].unique()
    for p1 in products:
        for p2 in products:
            if p1 != p2:
                G.add_edge(p1, p2)

In [ ]:
#Number of nodes
G.number_of_nodes()

30107

In [ ]:
#Number of links
G.number_of_edges()

21160

**CREATION OF THE TRANSITION MATRIX**

In [ ]:
#Creation of the transition matrix from the graph
TM = nx.to_numpy_array(G)

In [ ]:
#Square matrix with nodes ad indexes 
np.shape(TM)

(30107, 30107)

In [ ]:
#Creating a function to check if our matrix is column-wise 
def check_col_sums(matrix):
    num_non_normalized = 0
    for col in range(matrix.shape[1]):
        col_sum = matrix[:,col].sum()
        if col_sum != 1:
            num_non_normalized += 1
    return num_non_normalized

check_col_sums(TM)

25465

In [ ]:
#The matrix is not column-wise
#Checking for dead ends and counting them
len(list(nx.isolates(G)))

19767

**DELETING DEAD ENDS AS FIRST APPROACH TO COMPUTE THE PAGERANK ALGORITHM**

In [ ]:
#Creation of a new transition matrix by deleting dead ends from the original one
G1=G.copy()
G1.remove_nodes_from(list(nx.isolates(G1)))

In [ ]:
#Shape of the new matrix without dead ends
TM_no_dead_nodes = nx.to_numpy_array(G1)
np.shape(TM_no_dead_nodes)

(10340, 10340)

In [ ]:
#PageRank computation
pageRank1 = nx.pagerank(G1, alpha=0.85, personalization=None, max_iter=10000, tol=1e-06, nstart=None, weight='weight', dangling=None)

#Sorting products on the basis of the PageRank in decreasing order 
pr1_sorted = sorted(pageRank1.items(), key=lambda x: x[1], reverse=True)

#Showing the first ten products on the basis of their PageRank
for i, (prod, rank) in enumerate(pr1_sorted[:10]):
    id_correspondence = df.loc[df["product_id"] == prod]
    if not id_correspondence.empty:
        title = id_correspondence.iloc[0]["product_title"]
        category = id_correspondence.iloc[0]["product_category"]
        print("Category:",category, "-", title, " - PageRank value:",round(rank,5))


B004LLIKVU 0.0012053680529658893
B00IX1I3G6 0.0005109026440581102
B004RMK5QG 0.0003821600359552175
B00H09BB16 0.00037721043593187773
B00H9A60O4 0.0003447424330581667
B00MV9EL6M 0.00028812176497696785
B00MCLGAAO 0.00026349279562820497
B00IY7BNUW 0.00025911919717131545
B0009X29WK 0.0002545904503599395
B00LC9UU6C 0.0002476274321675711


**KEEPING DEAD ENDS AS SECOND APPROACH TO COMPUTE PAGE RANK**

In [ ]:
#PageRank computation without deleting dead nodes
pageRank2 = nx.pagerank(G, alpha=0.85, personalization=None, max_iter=10000, tol=1e-06, nstart=None, weight='weight', dangling=None)
pr2_sorted = sorted(pageRank2.items(), key=lambda x: x[1], reverse=True)

for i, (prod, rank) in enumerate(pr2_sorted[:10]):
    id_correspondence = df.loc[df["product_id"] == prod]
    if not id_correspondence.empty:
        title = id_correspondence.iloc[0]["product_title"]
        category = id_correspondence.iloc[0]["product_category"]
        print("Category:",category, "-", title, " - PageRank value:",round(rank,5))

B004LLIKVU 0.0009178036985504536
B00IX1I3G6 0.00038657596189404586
B004RMK5QG 0.0002897827692132318
B00H09BB16 0.00028178149344370977
B00H9A60O4 0.00023937027432919188
B00MV9EL6M 0.00021873705801196415
B00MCLGAAO 0.00019962589193164309
B00IY7BNUW 0.00019469902913515325
B0009X29WK 0.00019401191192324708
B004XNK7AI 0.00018821493846207153
